In [ ]:
tez_path = '../input/tez-lib/'
effnet_path = '../input/efficientnet-pytorch/'
timm_path = '../input/timm-pytorch-image-models/pytorch-image-models-master'
import sys
sys.path.append(tez_path)
sys.path.append(effnet_path)
sys.path.append(timm_path)

In [ ]:
import os
import albumentations
import pandas as pd
import numpy as np
import timm
import tez
from tez.datasets import ImageDataset

import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

In [ ]:
class EfficientnetModel(tez.Model):
    def __init__(self, num_classes):
        super().__init__()

        self.effnet = EfficientNet.from_name("efficientnet-b4")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1792, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, None
    
    def load(self, model_path, device="cuda"):
        self.device = device
        if next(self.parameters()).device != self.device:
            self.to(self.device)
        model_dict = torch.load(model_path, map_location=device)
        self.load_state_dict(model_dict["state_dict"])
        
    def predict(self, dataset, sampler=None, batch_size=16, n_jobs=1, collate_fn=None):
        if next(self.parameters()).device != self.device:
            self.to(self.device)

        if batch_size == 1:
            n_jobs = 0
        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, num_workers=4, sampler=sampler, collate_fn=collate_fn, pin_memory=True
        )

        if self.training:
            self.eval()

        final_output = []
        tk0 = tqdm(data_loader, total=len(data_loader))

        for b_idx, data in enumerate(tk0):
            with torch.no_grad():
                out = self.predict_one_step(data)
                out = out.cpu().detach().numpy()
                yield out

            tk0.set_postfix(stage="test")
        tk0.close()
        
    def model_fn(self, data):
        for key, value in data.items():
            data[key] = value.to(self.device)
        #if self.fp16:
        #    with torch.cuda.amp.autocast():
        #        output, loss, metrics = self(**data)
        #else:
        output, loss, metrics = self(**data)
        return output, loss, metrics
    
    def predict_one_step(self, data):
        output, _, _ = self.model_fn(data)
        return output

In [ ]:
class CustomResNext(tez.Model):
    def __init__(self, num_classes, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()

        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.out = nn.Linear(n_features, num_classes)
        self.step_scheduler_after = "epoch"

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape
        outputs = self.model(image)
        return outputs, None, None


In [ ]:
# augmentations taken from: https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-inference-tta
img_size = 512
test_aug = albumentations.Compose([
    albumentations.RandomResizedCrop(img_size, img_size),
    albumentations.Transpose(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HueSaturationValue(
        hue_shift_limit=0.2, 
        sat_shift_limit=0.2,
        val_shift_limit=0.2, 
        p=0.5
    ),
    albumentations.RandomBrightnessContrast(
        brightness_limit=(-0.1,0.1), 
        contrast_limit=(-0.1, 0.1), 
        p=0.5
    ),
    albumentations.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225], 
        max_pixel_value=255.0, 
        p=1.0
    )
], p=1.)

In [ ]:
dfx = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
image_path = "../input/cassava-leaf-disease-classification/test_images/"
test_image_paths = [os.path.join(image_path, x) for x in dfx.image_id.values]
# fake targets
test_targets = dfx.label.values
test_dataset = ImageDataset(
    image_paths=test_image_paths,
    targets=test_targets,
    #resize=None,
    augmentations=test_aug,
)

In [ ]:
train_dfx = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
model_path = "../input/cassava-model-3"

model0 = EfficientnetModel(num_classes=train_dfx.label.nunique())
model0.load(f"{model_path}/efficentnet_model_fold0.bin", device='cuda')

model1 = EfficientnetModel(num_classes=train_dfx.label.nunique())
model1.load(f"{model_path}/efficentnet_model_fold1.bin", device='cuda')

model2 = EfficientnetModel(num_classes=train_dfx.label.nunique())
model2.load(f"{model_path}/efficentnet_model_fold2.bin", device='cuda')

model3 = EfficientnetModel(num_classes=train_dfx.label.nunique())
model3.load(f"{model_path}/efficentnet_model_fold3.bin", device='cuda')

model4 = EfficientnetModel(num_classes=train_dfx.label.nunique())
model4.load(f"{model_path}/efficentnet_model_fold4.bin", device='cuda')


model5= CustomResNext(num_classes=train_dfx.label.nunique())
model5.load(f"{model_path}/resnet_model_fold0.bin", device='cuda')

model6= CustomResNext(num_classes=train_dfx.label.nunique())
model6.load(f"{model_path}/resnet_model_fold1.bin", device='cuda')

model7= CustomResNext(num_classes=train_dfx.label.nunique())
model7.load(f"{model_path}/resnet_model_fold2.bin", device='cuda')

model8= CustomResNext(num_classes=train_dfx.label.nunique())
model8.load(f"{model_path}/resnet_model_fold3.bin", device='cuda')

model9= CustomResNext(num_classes=train_dfx.label.nunique())
model9.load(f'{model_path}/resnet_model_fold4.bin', device='cuda')


In [ ]:
from scipy.special import softmax
model_list = [model0, model1, model2, model3, model4, model5, model6, model7, model8, model9]

def run_inference(model):
    final_preds = None
    for j in range(7):
        preds = model.predict(test_dataset, batch_size=64, n_jobs=-1)
        temp_preds = None
        for p in preds:
            if temp_preds is None:
                temp_preds = p
            else:
                temp_preds = np.vstack((temp_preds, p))
        if final_preds is None:
            final_preds = temp_preds
        else:
            final_preds += temp_preds
    final_preds /= 7
    final_prob = softmax(final_preds)
    final_preds = final_preds.argmax(axis=1)
    return final_preds, final_prob[0][final_preds]


In [ ]:
new_df = pd.DataFrame()
new_df['model0'], new_df['model0_prob'] = run_inference(model_list[0])
new_df['model1'], new_df['model1_prob'] = run_inference(model_list[1])
new_df['model2'], new_df['model2_prob'] = run_inference(model_list[2])
new_df['model3'], new_df['model3_prob'] = run_inference(model_list[3])
new_df['model4'], new_df['model4_prob'] = run_inference(model_list[4])
new_df['model5'], new_df['model5_prob'] = run_inference(model_list[5])
new_df['model6'], new_df['model6_prob'] = run_inference(model_list[6])
new_df['model7'], new_df['model7_prob'] = run_inference(model_list[7])
new_df['model8'], new_df['model8_prob'] = run_inference(model_list[8])
new_df['model9'], new_df['model9_prob'] = run_inference(model_list[9])


In [ ]:
from collections import Counter
def most_common(lst):
    data = Counter(lst)
    return max(lst, key=data.get)

final_preds = list()
for index, row in new_df.iterrows():
    out_list = [row['model0'], row['model1'], row['model2'], row['model3'], row['model4'],row['model5'], row['model6'], row['model7'], row['model8'], row['model9'] ]
    final_preds.append(most_common(out_list))

In [ ]:
"""
#Approach 3 for assemble
from statistics import mode
final_preds = list()

for index, row in new_df.iterrows():
    temp_df = pd.DataFrame()
    temp_df['model'] = [row['model0'], row['model1'], row['model2'], row['model3'], row['model4'],row['model5'], row['model6'], row['model7'], row['model8'], row['model9'] ]
    temp_df['model_prob'] = [row['model0_prob'], row['model1_prob'], row['model2_prob'], row['model3_prob'], row['model4_prob'],row['model5_prob'], row['model6_prob'], row['model7_prob'], row['model8_prob'], row['model9_prob'] ]
    out_df = temp_df.groupby('model').sum().sort_values('model_prob', ascending=False).reset_index()
    pred = out_df['model'].tolist()[0]
    final_preds.append(pred)
    
"""
        

In [ ]:
"""
Approach 2
from scipy.special import softmax
def run_inference(model):
    final_preds = None
    for j in range(5):
        preds = model.predict(test_dataset, batch_size=64, n_jobs=-1)
        temp_preds = None
        for p in preds:
            if temp_preds is None:
                temp_preds = p
            else:
                temp_preds = np.vstack((temp_preds, p))
        if final_preds is None:
            final_preds = temp_preds
        else:
            final_preds += temp_preds
    final_preds /= 5
    return final_preds
    
    
out_preds1 = np.empty([1,5])
out_preds2 = np.empty([1,1000])
for model in model_list:
    try:
        out_preds1 = np.add(out_preds1, run_inference(model))
    except Exception as ex:
        out_preds2 = np.add(out_preds2, run_inference(model))
"""

In [ ]:
#softmax(out_preds1).argmax(axis=1)
#out = 0.5*out_preds1 + 0.5*out_preds2[:,:5]
#final_preds = softmax(out).argmax(axis=1)
#out_predsoft1 = softmax(out_preds1)
#out_predsoft2 = softmax(out_preds2[:,:5])

#out = np.add(out_predsoft1, out_predsoft2)
#final_preds = softmax(out).argmax(axis=1)

In [ ]:
dfx.label = final_preds
dfx.to_csv("submission.csv", index=False)